# New sequences creation, LSTM+CNN with multiple district outputs

In [51]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout,
    Input, InputLayer, concatenate, GlobalMaxPooling2D, Reshape
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.ndimage import gaussian_filter
import cv2
import pickle

In [52]:
df = pd.read_csv(r"/home/scc/miranda.barros-everett/chicago_with_census_2018_2024.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7800353 entries, 0 to 7800352
Data columns (total 38 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Date                           object 
 1   IUCR                           object 
 2   Primary Type                   object 
 3   Latitude                       float64
 4   Longitude                      float64
 5   Month                          int64  
 6   Season                         object 
 7   Year                           int64  
 8   District                       int64  
 9   Beat                           int64  
 10  geometry                       object 
 11  Community Name                 object 
 12  Community Area Number          int64  
 13  NIBRS                          object 
 14  Description                    object 
 15  Category                       object 
 16  Crime Against Category         object 
 17  district_name                  object 
 18  te

In [53]:
#filter year(s)
df = df.loc[df['Year'].isin([2018, 2019]), :] #18 und 19

In [54]:
# Count unique values in 'district'
df_num_unique_districts = df['District'].nunique()
print(f"Number of unique districts: {df_num_unique_districts}")

# Count occurrences of each unique value in 'district'
df_district_counts = df['District'].value_counts()
print("Counts of each district:")
print(df_district_counts)

Number of unique districts: 23
Counts of each district:
11    36109
6     31722
8     30737
18    30541
1     29954
4     26414
7     26395
12    25182
25    25139
10    23968
3     23386
19    23336
2     21911
5     21554
9     20932
15    19206
14    17970
16    16359
22    15774
24    15084
17    13866
20     9017
31       18
Name: District, dtype: int64


In [55]:
# Assuming your DataFrame is named 'df'
df = df[df['District'] != 31]

In [56]:
# Count unique values in 'district'
df_num_unique_districts = df['District'].nunique()
print(f"Number of unique districts: {df_num_unique_districts}")

# Count occurrences of each unique value in 'district'
df_district_counts = df['District'].value_counts()
print("Counts of each district:")
print(df_district_counts)

Number of unique districts: 22
Counts of each district:
11    36109
6     31722
8     30737
18    30541
1     29954
4     26414
7     26395
12    25182
25    25139
10    23968
3     23386
19    23336
2     21911
5     21554
9     20932
15    19206
14    17970
16    16359
22    15774
24    15084
17    13866
20     9017
Name: District, dtype: int64


In [57]:
#clean column names
df.columns
df.rename(columns={
    'Date': 'time_stamp',    
    'Hour': 'hour',
    'DayOfWeek': 'day_of_week',
    'Month': 'month',
    'Season': 'season',
    'Year': 'year',
    
    'Primary Type': 'primary_type',
    'Description': 'description',
    'Category':'category',
    'Crime Against Category':'crime_against_category', 
    
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'District':'district', 
    'Beat': 'beat',
    'Community Name': 'community_name',
    'Community Area Number': 'community_area_number',
    
    'TOT_POP': 'tot_pop',
    'UNEMP': 'unemp',
    'INC_LT_25K': 'inc_lt_25k'
}, inplace=True)

#clean datatypes
print(df.dtypes)

df['time_stamp'] = pd.to_datetime(df['time_stamp'])
categorical_columns = ['season', 'primary_type', 'category', 'crime_against_category', 'community_name']
df[categorical_columns] = df[categorical_columns].astype('category')

#new variable: date
df['date'] = df['time_stamp'].dt.date
df['date'] = pd.to_datetime(df['date'])

#new variable: week_in_year
df['week_in_year'] = df['date'].dt.strftime('%Y-%U')

time_stamp                        object
IUCR                              object
primary_type                      object
latitude                         float64
longitude                        float64
month                              int64
season                            object
year                               int64
district                           int64
beat                               int64
geometry                          object
community_name                    object
community_area_number              int64
NIBRS                             object
description                       object
category                          object
crime_against_category            object
district_name                     object
temperature_2m_max_day           float64
temperature_2m_min_day           float64
apparent_temperature_mean_day    float64
daylight_duration_day            float64
sunshine_duration_day            float64
precipitation_sum_day            float64
rain_sum_day    

In [58]:
#preparation heatmap: group data by week and district
num_features = [
    'temperature_2m_max_day', 'temperature_2m_min_day', 'apparent_temperature_mean_day',
    'daylight_duration_day', 'sunshine_duration_day', 'precipitation_sum_day',
    'rain_sum_day', 'snowfall_sum_day', 'precipitation_hours_day', 
    'apparent_temperature_hour', 'precipitation_hour', 'rain_hour', 
    'snowfall_hour', 'cloud_cover_hour', 'wind_speed_100m_hour', 
    'tot_pop', 'unemp', 'inc_lt_25k'
]

#aggregation of the data by district and week, and calculation of the median
df['crimes_per_week'] = 1  #add a helper column to count the crimes

#aggregation of the data by district and week, and calculation of the median
data = df.groupby(['district', 'week_in_year']).agg(
    total_crimes=('crimes_per_week', 'sum'),  #count the number of crimes
    **{var: (var, 'median') for var in num_features}  #calculate the median of numerical features
).reset_index()

#output the aggregated df
print(data.head())  # Print the first few rows to check sorting
print(data.tail())  # Print the last few rows to ensure it's sorted as expected

   district week_in_year  total_crimes  temperature_2m_max_day  \
0         1      2018-00           219              -11.143499   
1         1      2018-01           279                4.006500   
2         1      2018-02           259               -2.793500   
3         1      2018-03           292                6.856500   
4         1      2018-04           293               -0.343500   

   temperature_2m_min_day  apparent_temperature_mean_day  \
0              -17.693500                     -21.635940   
1               -4.293500                      -5.290957   
2              -12.643499                     -14.063271   
3                0.406500                      -0.672545   
4               -7.293500                     -10.021296   

   daylight_duration_day  sunshine_duration_day  precipitation_sum_day  \
0              33273.793             28849.9880                    0.0   
1              33691.383              5621.2524                    0.5   
2              34341

In [35]:
def create_heatmap_array(data, map_size=(256, 256), radius=5):
    """
    creates a heatmap as a numpy array based on latitude and longitude
    
    parameters:
    - data: df with the columns 'latitude', 'longitude', 'crimes_per_week'
    - map_size: size of the heatmap (height, width)
    - radius: radius of the heatmap points
    
    returns:
    - heatmap_array: numpy array of the specified map_size
    """
    #initialize array
    heatmap_array = np.zeros(map_size)

    #normalize latitude and longitude to map size
    lat_min, lat_max = data['latitude'].min(), data['latitude'].max()
    lon_min, lon_max = data['longitude'].min(), data['longitude'].max()

    #check if lat_max equals lat_min or lon_max equals lon_min
    if lat_max == lat_min:
        lat_max += 0.001  #adjust slightly to avoid zero division
    if lon_max == lon_min:
        lon_max += 0.001  #adjust slightly to avoid zero division

    #scaling function
    def scale_lat_lon(lat, lon):
        x = int((lon - lon_min) / (lon_max - lon_min) * (map_size[1] - 1))
        y = int((lat - lat_min) / (lat_max - lat_min) * (map_size[0] - 1))
        return x, y

    for _, row in data.iterrows():
        x, y = scale_lat_lon(row['latitude'], row['longitude'])
        heatmap_array[y, x] += row['crimes_per_week']

    return heatmap_array

# create a heatmap for a week and a district, and resize it to 256x256
def create_and_resize_heatmap(week_data, map_size=(256, 256), radius=5):
    heatmap_array = create_heatmap_array(week_data, map_size=map_size, radius=radius)
    heatmap_resized = cv2.resize(heatmap_array, map_size)
    return heatmap_resized

# list of variables for which the median was calculated
num_features = [
    'temperature_2m_max_day', 'temperature_2m_min_day', 'apparent_temperature_mean_day',
    'daylight_duration_day', 'sunshine_duration_day', 'precipitation_sum_day',
    'rain_sum_day', 'snowfall_sum_day', 'precipitation_hours_day', 
    'apparent_temperature_hour', 'precipitation_hour', 'rain_hour', 
    'snowfall_hour', 'cloud_cover_hour', 'wind_speed_100m_hour', 
    'tot_pop', 'unemp', 'inc_lt_25k'
]

X_images_sequences: This will store the sequences of heatmaps (or images) for all districts, for the last window_size weeks.
X_socio_sequences: This will store the sequences of socio-economic data for all districts, for the last window_size weeks.
y_labels_sequences: This will store the crime counts for all districts, but only for the week following the window_size weeks.
For instance, if you have 23 districts and a window size of 10 weeks, then:

Each element in X_images_sequences will have shape (23, 10, 256, 256, 1).
Each element in X_socio_sequences will have shape (23, 10, num_features).
Each element in y_labels_sequences will be a 1D array of size 23, where each value corresponds to the crime count for each district in the week following the 10-week window.
Example:
Let's say you're processing data for district 1, and your current window_size is 10:

X_images_sequences[i]: Contains the heatmap data for district 1 across the last 10 weeks.
X_socio_sequences[i]: Contains the socio-economic data for district 1 across the last 10 weeks.
y_labels_sequences[i]: Contains the crime count for district 1 in the week after these 10 weeks.
When combining all districts:

X_images_sequences[j]: Contains sequences for all 23 districts.
y_labels_sequences[j]: Contains the crime counts for all 23 districts for the week following the sequence.
So, after you complete the sequence generation for all districts and all time windows, y_labels_sequences will hold the crime counts for every district for the relevant weeks, allowing you to predict and compare the actual crime counts per district.

In [36]:
# Double checking district number stays the same
districts = data['district'].unique()
num_districts = len(districts)
print(districts)
print(num_districts)

[ 1  2  3  4  5  6  7  8  9 10 11 12 14 15 16 17 18 19 20 22 24 25]
22


In [38]:
districts = data['district'].unique()
# Initialize your lists
X_images_sequences = []
X_socio_sequences = []
y_labels_sequences = []
district_mapping = []  # This will store the district for each sequence

window_size = 10  # number of weeks in the sliding window
max_weeks = 159   # maximum number of weeks to be processed

# Iterate over all districts
for neighborhood in districts:
    neighborhood_data = data[data['district'] == neighborhood]
    neighborhood_data = neighborhood_data.sort_values('week_in_year')
    
    # Limit to the first XX weeks
    neighborhood_data = neighborhood_data.head(max_weeks)
    
    for i in range(len(neighborhood_data) - window_size):
        window_data = neighborhood_data.iloc[i:i + window_size]
        next_week_data = neighborhood_data.iloc[i + window_size]
        
        # Create heatmap sequence
        heatmaps_sequence = np.array([
            create_and_resize_heatmap(df[(df['district'] == neighborhood) & (df['week_in_year'] == week)])
            for week in window_data['week_in_year']
        ])
        
        # Create socio-economic feature sequence
        socio_sequence = window_data[num_features].values
        
        # Append sequences
        X_images_sequences.append(heatmaps_sequence)
        X_socio_sequences.append(socio_sequence)
        
        # Store the crime counts for all districts in the target week
        all_districts_crime_counts = data[data['week_in_year'] == next_week_data['week_in_year']]['total_crimes'].values
        y_labels_sequences.append(all_districts_crime_counts)
        
        # Track the district (optional if you need this mapping)
        district_mapping.append(neighborhood)  # Store the current district



In [39]:
# Save the new pickle file
save_path = r"/home/scc/miranda.barros-everett/model/2018_2019_with_district_data_final_without_d31.pkl"

with open(save_path, 'wb') as f:
    pickle.dump((X_images_sequences, X_socio_sequences, y_labels_sequences), f)

print(f"Data has been successfully saved to {save_path}.")

Data has been successfully saved to /home/scc/miranda.barros-everett/model/2018_2019_with_district_data_final_without_d31.pkl.


In [41]:
# Convert lists to numpy arrays
X_images_sequences = np.array(X_images_sequences)
X_socio_sequences = np.array(X_socio_sequences)

# Check the length of sequences in y_labels_sequences
#sequence_lengths = [len(seq) for seq in y_labels_sequences]
#print("Sequence lengths in y_labels_sequences:", sequence_lengths)

# Determine the maximum length
#max_length = max(sequence_lengths)
#print("Maximum length of sequences:", max_length)

# Pad y_labels_sequences to ensure consistent shape
#padded_y_labels_sequences = np.array([
#    np.pad(seq, (0, max_length - len(seq)), mode='constant', constant_values=0) if len(seq) < max_length
#    else seq
#    for seq in y_labels_sequences
#])

# Print shapes of the data arrays
print("Shapes before normalization:")
print("X_images_sequences shape:", X_images_sequences.shape)
print("X_socio_sequences shape:", X_socio_sequences.shape)
print("y_labels_sequences shape:", padded_y_labels_sequences.shape)

# Normalize the socio-economic data
scaler_socio = StandardScaler()
X_socio_sequences = scaler_socio.fit_transform(X_socio_sequences.reshape(-1, X_socio_sequences.shape[-1])).reshape(X_socio_sequences.shape)

# Normalize the crime counts for each district separately
scaler_y_labels = StandardScaler()
# Reshape padded_y_labels_sequences to 2D (number of samples x number of districts)
y_labels_flat = padded_y_labels_sequences.reshape(-1, padded_y_labels_sequences.shape[-1])
y_labels_normalized = scaler_y_labels.fit_transform(y_labels_flat)

# Reshape back to the original 3D shape (number of samples x window_size x number of districts)
y_labels_sequences = y_labels_normalized.reshape(padded_y_labels_sequences.shape)

# Print shapes after normalization
print("Shapes after normalization:")
print("X_images_sequences shape:", X_images_sequences.shape)
print("X_socio_sequences shape:", X_socio_sequences.shape)
print("y_labels_sequences shape:", y_labels_sequences.shape)

# Prepare data for training
X_images_sequences = X_images_sequences.reshape((X_images_sequences.shape[0], window_size, 256, 256, 1))

# Split the data
X_images_train, X_images_test, X_socio_train, X_socio_test, y_train, y_test = train_test_split(
    X_images_sequences, X_socio_sequences, y_labels_sequences, test_size=0.1, random_state=42
)

# Print shapes after splitting
print("Shapes after train/test split:")
print("X_images_train shape:", X_images_train.shape)
print("X_images_test shape:", X_images_test.shape)
print("X_socio_train shape:", X_socio_train.shape)
print("X_socio_test shape:", X_socio_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Shapes before normalization:
X_images_sequences shape: (2112, 10, 256, 256, 1)
X_socio_sequences shape: (2112, 10, 18)
y_labels_sequences shape: (2112, 22)
Shapes after normalization:
X_images_sequences shape: (2112, 10, 256, 256, 1)
X_socio_sequences shape: (2112, 10, 18)
y_labels_sequences shape: (2112, 22)
Shapes after train/test split:
X_images_train shape: (1900, 10, 256, 256, 1)
X_images_test shape: (212, 10, 256, 256, 1)
X_socio_train shape: (1900, 10, 18)
X_socio_test shape: (212, 10, 18)
y_train shape: (1900, 22)
y_test shape: (212, 22)


In [42]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, LSTM, Dropout, Dense, concatenate, BatchNormalization, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

# Model architecture
input_images = Input(shape=(window_size, 256, 256, 1))

# CNN layers with batch normalization 
x = TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))(input_images)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
x = TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))(x)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
x = TimeDistributed(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)))(x)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
x = TimeDistributed(Flatten())(x)

# LSTM layers on image features
x = LSTM(50, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.01))(x)
x = LSTM(50, activation='tanh', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x) 

input_socio = Input(shape=(window_size, X_socio_train.shape[2]))

# LSTM layer on socio-economic data 
y = LSTM(50, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.01))(input_socio)
y = LSTM(50, activation='tanh', kernel_regularizer=l2(0.01))(y)
y = Dropout(0.5)(y)  

combined = concatenate([x, y])

# Fully connected layers with dropout and L2 regularization
z = Dense(50, activation='relu', kernel_regularizer=l2(0.01))(combined)
z = Dropout(0.5)(z)  

# Output layer modified to predict per district
#output = Dense(num_districts)(z)
#output = Dense(num_districts, activation='relu')(z)
# alternative two:
output = Dense(num_districts, activation='softplus')(z)

# Compile the model
model = Model(inputs=[input_images, input_socio], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')

# Summarize the model
model.summary()

# Callbacks for dynamic learning rate adjustment and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Training
history = model.fit(
    [X_images_train, X_socio_train], y_train,
    validation_data=([X_images_test, X_socio_test], y_test),
    epochs=100,
    batch_size=32,
    callbacks=[reduce_lr, early_stopping]
)

# Save the model to a specific folder
model.save(r"/home/scc/miranda.barros-everett/model/saved_models/all_crimes_model_2018_2019_final.h5")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 10, 256, 256, 1)]    0         []                            
                                                                                                  
 time_distributed (TimeDist  (None, 10, 256, 256, 32)     320       ['input_1[0][0]']             
 ributed)                                                                                         
                                                                                                  
 time_distributed_1 (TimeDi  (None, 10, 256, 256, 32)     128       ['time_distributed[0][0]']    
 stributed)                                                                                       
                                                                                              

In [44]:
print("Shape of y_test:", y_test.shape)
print("Shape of y_pred_sequences:", y_pred.shape)

Shape of y_test: (212, 22)
Shape of y_pred_sequences: (212, 22)


In [43]:
y_pred = model.predict([X_images_test, X_socio_test])

7/7 [==============================] - 6s 545ms/step


In [46]:
# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Inverse-transform predictions and true values
#y_test_original = scaler_y_labels.inverse_transform(y_test)
#y_pred_original = scaler_y_labels.inverse_transform(y_pred)

# Calculate MSE on the original scale
mse_original = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on original scale:", mse_original)

from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print("R-squared Score:", r2)

print("Mean of y_test:", np.mean(y_test))
print("Mean of y_pred:", np.mean(y_pred))

print("Variance of y_test:", np.var(y_test))
print("Variance of y_pred:", np.var(y_pred))

Mean Squared Error: 0.649365800199699
Mean Squared Error on original scale: 0.649365800199699
R-squared Score: 0.19914357217676348
Mean of y_test: -5.767477552744513e-05
Mean of y_pred: 0.25377318
Variance of y_test: 0.8133213429903979
Variance of y_pred: 0.08802336


In [47]:
num_samples_to_show = 1  # Number of samples to display
print("True Values (First few samples):")
print(y_test[:num_samples_to_show])

print("Predicted Values (First few samples):")
print(y_pred[:num_samples_to_show])

True Values (First few samples):
[[-0.04041443  1.17526459  0.35156033  0.61473745  0.78182218  1.46636315
   0.7436697   0.22155504  0.10782744  1.06977434  1.41464314  0.54328675
   0.56386732  0.8770466   1.13073047 -0.4658522   0.30439755 -0.17840121
   0.2602474   1.50456844  1.64911085  1.03045172]]
Predicted Values (First few samples):
[[0.64266753 0.7280608  0.6890951  0.6994651  0.7097076  0.749153
  0.73813343 0.6695309  0.6905838  0.6551253  0.6509831  0.66295606
  0.6328733  0.6894588  0.54757524 0.57337064 0.61229163 0.658701
  0.63499624 0.683954   0.7019525  0.6062955 ]]


In [49]:
# Unnormalize predictions
y_pred_flat = y_pred.reshape(-1, y_pred.shape[-1])
y_pred_original = scaler_y_labels.inverse_transform(y_pred_flat)
y_pred_sequences = y_pred_original.reshape(y_pred.shape)

# Unnormalize true values
y_test_flat = y_test.reshape(-1, y_test.shape[-1])
y_test_original = scaler_y_labels.inverse_transform(y_test_flat)
y_test_sequences = y_test_original.reshape(y_test.shape)

# Print some values for comparison
num_samples_to_show = 1 # Number of samples to display

print("Unnormalized True Values (First few samples):")
print(y_test_sequences[:num_samples_to_show])

print("Unnormalized Predicted Values (First few samples):")
print(y_pred_sequences[:num_samples_to_show])

Unnormalized True Values (First few samples):
[[282. 252. 236. 275. 233. 371. 286. 301. 203. 266. 422. 264. 190. 210.
  180. 120. 305. 216.  90. 186. 187. 273.]]
Unnormalized Predicted Values (First few samples):
[[317.0363  235.43974 248.55138 278.17004 230.57584 338.20938 285.75394
  319.32404 221.91585 251.66672 380.34912 268.8092  192.10596 204.29861
  166.54796 143.1213  319.07654 249.04123  96.62987 166.45027 162.33907
  258.6147 ]]
